<a href="https://colab.research.google.com/github/Shaik1903/english-to-telugu-translator/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
Sentence Embedding Function
Replace with the HF tokernizers or create one
'''

'\nSentence Embedding Function\nReplace with the HF tokernizers or create one\n'

In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
english_file = '/content/drive/MyDrive/Projects/English to Telugu Translator/train.en'
telugu_file = '/content/drive/MyDrive/Projects/English to Telugu Translator/train.te'

In [4]:
START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]


In [5]:
telugu_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                     '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                     'ఁ', 'ఆ', 'ఇ', 'ా', 'ి', 'ీ', 'ు', 'ూ',
                     'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ౠ', 'ఌ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ',
                     'క', 'ఖ', 'గ', 'ఘ', 'ఙ',
                     'చ', 'ఛ', 'జ', 'ఝ', 'ఞ',
                     'ట', 'ఠ', 'డ', 'ఢ', 'ణ',
                     'త', 'థ', 'ద', 'ధ', 'న',
                     'ప', 'ఫ', 'బ', 'భ', 'మ',
                     'య', 'ర', 'ల', 'వ', 'శ', 'ష', 'స', 'హ', 'క్ష', 'ళ', 'ఱ',
                     '఼', 'ఽ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ౄ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', '్', 'ౕ', 'ౖ', '౞', 'ౣ', 'ం', 'ః',
                     '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯', PADDING_TOKEN, END_TOKEN]


In [6]:
index_to_telugu = {k:v for k,v in enumerate(telugu_vocabulary)}
telugu_to_index = {v:k for k,v in enumerate(telugu_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [7]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(telugu_file, 'r') as file:
    telugu_sentences = file.readlines()

In [8]:
# Limit Number of sentences
TOTAL_SENTENCES = 4300
english_sentences = english_sentences[:TOTAL_SENTENCES]
telugu_sentences = telugu_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]   # This removes the newline character and converts all of the words into lower case
telugu_sentences = [sentence.rstrip('\n') for sentence in telugu_sentences]

In [9]:
english_sentences[-5:]

['belgian congolese forces under the command of belgian officers notably fought against the italian colonial army in ethiopia in asosa, borta and sao under major-general auguste-eduard gilliaert during the second east african campaign.',
 'dont slog too much',
 'doctors advised him for a few days rest.',
 '( 1) foreseeing dangerous situations and avoiding them.',
 'this implies an argument of perigee of either 0 or 180.']

In [10]:
telugu_sentences[-5:]

['బెల్జియన్ అధికారుల ఆధ్వర్యంలో బెల్జియన్ కాంగో దళాలు ప్రత్యేకంగా రెండో తూర్పు ఆఫ్రికన్ పోరాటసమయంలో మేజర్-జనరల్ అగస్టే-ఎడార్డ్ గిల్లియాట్, సాయో నేతృత్వంలోని అస్సోసాలో (ఇథియోపియాలోని) ఇటాలియన్ వలస సైన్యంపై పోరాడారు.',
 'ఎక్కువ బ్రష్ చేయకండి',
 'కొన్ని రోజులపాటు విశ్రాంతి అవసరమని వైద్యులు తెలిపారు.',
 '( 1) ప్రమాదాలను ముందే పసిగట్టి తప్పించుకోవడం.',
 'అంటే దానర్థం, పెరిజీ ఆర్గ్యుమెంటు 0 డిగ్రీలు గాని, 180 డిగ్రీలుగానీ ఉండాలి.']

In [11]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for i in range(len(telugu_sentences)):
    telugu_sentence, english_sentence = telugu_sentences[i], english_sentences[i]
    if is_valid_length(telugu_sentence, max_sequence_length) and is_valid_length(english_sentence, max_sequence_length) and is_valid_tokens(telugu_sentence, telugu_vocabulary):
        valid_sentence_indicies.append(i)

print(f"Number of sentences: {len(telugu_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 4300
Number of valid sentences: 3275


In [ ]:
telugu_sentences = [telugu_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [13]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, telugu_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = telugu_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]

In [14]:
dataset = TextDataset(english_sentences, telugu_sentences)

In [15]:
len(dataset)

4300

In [16]:
dataset[1]

('how do we glorify jehovahs undeserved kindness?',
 'యెహోవా కృపను మనమెలా మహిమపరచవచ్చు?')

In [17]:
batch_size = 3
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [19]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 1:
        break

[('in monaco-ville, street signs are printed in both french and mongasque.', 'the same has been confirmed in a report carried by bbc.', "i said, 'who are you?'"), ('మొనాకో-విల్లెలో, ఫ్రెంచ్, మోనెగస్క్యూ రెండింటిలో వీధి చిహ్నాలను ప్రింట్ చేస్తారు.', 'ప్రముఖ బీబీసీ నివేదిక ఇదే విషయం పేర్కొంది.', "'ఎవరు మీరు' అన్నాను.")]
[('in this state they can easily survive one year or longer.', 'the complete details of his death are yet to be known.', 'if you go...'), ('సంవత్సరానికి అదనంగా ఒక రోజునో లేక ఒక నెలనో చేర్చి ఈ తేడాను నివారించవచ్చు.', 'వీరి మరణానికి సంబంధించి పూర్తి వివరాలు ఇంకా తెలియాల్సి ఉంది.', 'రథము వెడలె .')]
[('"all the very best."""', 'what is this, child?', 'speaking during another session on reducing poverty through social and economic transformation, shri gangwar the government of india, under the leadership of prime minister shri narendra modi, is committed to eradicate poverty and promote prosperity in a changing world'), ('ఆల్ ది బెస్ట్\u200c”అని అన్నారు.', 'ఏంటిది చిన్న పిల్లా

In [20]:

def tokenize(sentence, language_to_index, start_token=True, end_token=True):
    sentence_word_indicies = [language_to_index[token] for token in list(sentence)]
    if start_token:
        sentence_word_indicies.insert(0, language_to_index[START_TOKEN])
    if end_token:
        sentence_word_indicies.append(language_to_index[END_TOKEN])
    for _ in range(len(sentence_word_indicies), max_sequence_length):
        sentence_word_indicies.append(language_to_index[PADDING_TOKEN])
    return torch.tensor(sentence_word_indicies)

In [21]:
batch

[('"all the very best."""',
  'what is this, child?',
  'speaking during another session on reducing poverty through social and economic transformation, shri gangwar the government of india, under the leadership of prime minister shri narendra modi, is committed to eradicate poverty and promote prosperity in a changing world'),
 ('ఆల్ ది బెస్ట్\u200c”అని అన్నారు.',
  'ఏంటిది చిన్న పిల్లాడిలా?',
  'సామాజిక, ఆర్థిక పరివర్తన ద్వారా పేదరికాన్ని తగ్గించడంపై మరో సెషన్\u200cలో మాట్లాడుతూ, ప్రధానమంత్రి శ్రీ నరేంద్రమోదీ నాయకత్వంలో భారత ప్రభుత్వం పేదరిక నిర్మూలనకు మరియు మారుతున్న ప్రపంచంలో శ్రేయస్సును ప్రోత్సహించడానికి కట్టుబడి ఉందని\xa0\xa0శ్రీ గంగ్వార్ అన్నారు.')]

In [24]:
'''ఇది ఒక ఉదాహరణ'''
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
# Example sentence
sentence = "సామాజిక, ఆర్థిక పరివర్తన ద్వారా పేదరికాన్ని తగ్గించడంపై మరో సెషన్\u200cలో మాట్లాడుతూ, ప్రధానమంత్రి శ్రీ నరేంద్రమోదీ నాయకత్వంలో భారత ప్రభుత్వం పేదరిక నిర్మూలనకు మరియు మారుతున్న ప్రపంచంలో శ్రేయస్సును ప్రోత్సహించడానికి కట్టుబడి ఉందని\xa0\xa0శ్రీ గంగ్వార్ అన్నారు."

# Encode the sentence with start and end tokens (no [CLS] token), and padding
encoded_dict = tokenizer.encode_plus(
    sentence,
    add_special_tokens=True,  # Add [SEP] token
    max_length=200,           # Set the maximum length to 512
    pad_to_max_length=True,   # Pad the sequence to the maximum length
    return_attention_mask=True,  # Generate attention mask
    return_tensors='pt',      # Return PyTorch tensors
    cls_token=None,           # Do not add the [CLS] token
    truncation=True           # Truncate or pad to the specified max_length
)

# Access the encoded tokens and attention mask
input_ids = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask']

# Print the encoded tokens and attention mask
print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)


In [28]:
def tokeniz(sentence,special_tokens = False):
  encoded_dict = tokenizer.encode_plus(
    sentence,
    add_special_tokens=special_tokens,  # Add [SEP] token
    max_length=200,           # Set the maximum length to 512
    pad_to_max_length=True,   # Pad the sequence to the maximum length
    return_attention_mask=True,  # Generate attention mask
    return_tensors='pt',      # Return PyTorch tensors
    cls_token=None,           # Do not add the [CLS] token
    truncation=True           # Truncate or pad to the specified max_length
    )
  input_ids = encoded_dict['input_ids']
  return input_ids

In [29]:
eng_tokenized, tl_tokenized = [], []
for sentence_num in range(batch_size):
    eng_sentence, tl_sentence = batch[0][sentence_num], batch[1][sentence_num]
    eng_tokenized.append( tokeniz(eng_sentence, False) )
    tl_tokenized.append( tokeniz(tl_sentence,True) )
eng_tokenized = torch.stack(eng_tokenized)
tl_tokenized = torch.stack(tl_tokenized)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Keyword arguments {'cls_token': None} not recognized.
Keyword arguments {'cls_token': None} not recognized.
Keyword arguments {'cls_token': None} not recognized.
Keyword arguments {'cls_token': None} not recognized.
Keyword arguments {'cls_token': None} not recognized.
Keyword arguments {'cls_token': None} not recognized.


In [31]:
tl_tokenized

tensor([[[   101,   1189,  15959,  35764,   1222,  72799,  20147,    100,
           29030,   1188,  37218,  27374,    119,    102,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,     

In [ ]:
# Loading Dataset
# Create Dataloaders

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

In [ ]:
def self_attention(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    print(f"scaled.size() : {scaled.size()}")
    if mask is not None:
        scaled = scaled.permute(1, 0, 2, 3) + mask
        scaled = scaled.permute(1, 0, 2, 3)
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, max_sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = self_attention(q, k, v, mask)
        values = values.permute(0,2,1,3).reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim)
        out = self.linear_layer(values)
        return out

In [ ]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y  + self.beta
        return out

In [ ]:
class FeedForwardNetwork(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(FeedForwardNetwork, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = FeedForwardNetwork(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, self_attention_mask):
        residual_x = x.clone()
        x = self.attention(x, mask=self_attention_mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x.clone()
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x

In [ ]:
class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
        x, self_attention_mask  = inputs
        for module in self._modules.values():
            x = module(x, self_attention_mask)
        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)   # tokenizer part - need to correct
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def forward(self, x, self_attention_mask, start_token, end_token):
        x = self.sentence_embedding(x, start_token, end_token)
        x = self.layers(x, self_attention_mask)
        return x


In [ ]:
class SeqEncoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                     for _ in range(num_layers)])

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model)
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, y, mask):
        batch_size, sequence_length, d_model = x.size() # this comes from the Encoder outputs
        kv = self.kv_layer(x)                           # We use this to make key and value matrices
        q = self.q_layer(y)                             # This is the continuation outputs in the decoder layer
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)
        kv = kv.permute(0, 2, 1, 3)
        q = q.permute(0, 2, 1, 3)
        k, v = kv.chunk(2, dim=-1)
        values, attention = self_attention(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        return out

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = FeedForwardNetwork(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y.clone() # Residuals
        y = self.self_attention(y, mask=self_attention_mask)
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)

        _y = y.clone()
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)

        _y = y.clone()
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
        return y


In [ ]:
class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for module in self._modules.values():
            y = module(x, y, self_attention_mask, cross_attention_mask)
        return y

In [ ]:
class Decoder(nn.Module):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN) # Need to add the tokenizer
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self, x, y, self_attention_mask, cross_attention_mask, start_token, end_token):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y

In [ ]:
class Transformer(nn.Module):
    def __init__(self,
                d_model,
                ffn_hidden,
                num_heads,
                drop_prob,
                num_layers,
                max_sequence_length,
                kn_vocab_size,
                english_to_index,
                telugu_to_index,
                START_TOKEN,
                END_TOKEN,
                PADDING_TOKEN
                ):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, telugu_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = nn.Linear(d_model, kn_vocab_size)
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    def forward(self,
                x,
                y,
                encoder_self_attention_mask=None,
                decoder_self_attention_mask=None,
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False, # We should make this true
                dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        out = self.linear(out)
        return out

Mounted at /content/drive


['for travel.',
 'there was a body.',
 'bigg boss is one of the biggest reality tv shows in india.',
 'he said fresh dates of examinations would be notified later.',
 'madhavan, anjali also acted in the movie along with anushka.']

['రైడ్ కోసం.',
 'అక్కడ ఒక శవం కనబడింది.',
 'బిగ్ బాస్ అనేది వరల్డ్స్ బిగ్గెస్ట్ రియాలిటీ షో.',
 'మరల ఎప్పుడు పరీక్షలు నిర్వహిస్తామో అనేది త్వరలోనే వెల్లడిస్తాం అన్నారు.',
 'ఈ చిత్రంలో అజయ్, ఇంద్రజ ఓ జంటగా, నాగశౌర్య, సనా మక్బుల్ మరో జంటగా నటించారు.']

In [ ]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length telugu: {np.percentile([len(x) for x in telugu_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length telugu: 169.0
97th percentile length English: 174.0


Number of sentences: 4300000
Number of valid sentences: 3252177


In [ ]:
telugu_sentences[-10:]

['బి.ఎన్.సెహగల్',
 '30 మేర తగ్గింది.',
 'పచ్చిపాలు : 2 టీస్పూన్లు',
 'గుజరాత్ లో బిజెపికి కాంగ్రెస్ గట్టి పోటీ ఇస్తోంది.',
 'కుడివైపు తిను',
 'రైడ్ కోసం.',
 'అక్కడ ఒక శవం కనబడింది.',
 'బిగ్ బాస్ అనేది వరల్డ్స్ బిగ్గెస్ట్ రియాలిటీ షో.',
 'మరల ఎప్పుడు పరీక్షలు నిర్వహిస్తామో అనేది త్వరలోనే వెల్లడిస్తాం అన్నారు.',
 'ఈ చిత్రంలో అజయ్, ఇంద్రజ ఓ జంటగా, నాగశౌర్య, సనా మక్బుల్ మరో జంటగా నటించారు.']

In [ ]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 1024
num_heads = 4
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
kn_vocab_size = len(telugu_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          telugu_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)


NameError: ignored

In [ ]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=kannada_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():                    # Xavier initialization helps in better convergence during training.
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)               #helps to prevent vanishing or exploding gradients during training.

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# Checking whether individual functions working or not

In [ ]:
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 200
ffn_hidden = 2048
num_layers = 5

# encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

In [ ]:
look_ahead_mask = torch.full([max_sequence_length, max_sequence_length], True)
look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
look_ahead_mask

tensor([[False,  True,  True,  ...,  True,  True,  True],
        [False, False,  True,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ..., False,  True,  True],
        [False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False]])

In [ ]:
x = torch.randn( (batch_size, max_sequence_length, d_model) ) # includes positional encoding
out = encoder(x)

------- ATTENTION 1 ------
x.size(): torch.Size([30, 200, 512])
qkv.size(): torch.Size([30, 200, 1536])
qkv.size(): torch.Size([30, 200, 8, 192])
qkv.size(): torch.Size([30, 8, 200, 192])
q size: torch.Size([30, 8, 200, 64]), k size: torch.Size([30, 8, 200, 64]), v size: torch.Size([30, 8, 200, 64]), 
scaled.size() : torch.Size([30, 8, 200, 200])
values.size(): torch.Size([30, 8, 200, 64]), attention.size:torch.Size([30, 8, 200, 200]) 
values.size(): torch.Size([30, 200, 512])
out.size(): torch.Size([30, 200, 512])
------- DROPOUT 1 ------
------- ADD AND LAYER NORMALIZATION 1 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512])
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 200, 512])
------- ATTENTION 2 ------
x after first linear layer: torch.Size([30, 200, 2048])
x after activation: torch.Size([30, 200, 2048])
x after dropout: torch.Size([30, 200, 2048])
x after 2nd linear layer: torch.

In [ ]:
q = np.random.randn(L,dim)
k = np.random.randn(L,dim)
v = np.random.randn(L,dim)

In [ ]:
q

array([[ 1.12591538, -0.20853284, -1.43566048,  0.16828876, -1.78339133,
         1.75671274,  0.48905883,  0.68204429],
       [ 0.17048747, -1.00515842,  0.98210632, -0.22148951, -0.68260914,
        -1.96552464, -1.51216378, -1.09425526],
       [-0.35344463,  1.41034386,  0.83618775, -0.98560087, -0.97709445,
         1.32576395, -0.39526458,  2.8772897 ],
       [ 1.65734447,  0.48862427,  1.12218185,  0.18465231, -0.62060722,
         0.39032771, -1.01891697,  1.69876902]])

In [ ]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

In [ ]:
def self_attn(q,k,v,mask = None):
  temp = np.matmul(q,k.T)/math.sqrt(q[-1])
  sc = temp+mask
  return np.matmul(softmax(sc),v)


In [ ]:
# multihead attention

In [ ]:
batch_size = 2
seq = 5
in_dim = 512
out_dim = 512
x = torch.randn(batch_size,seq,in_dim)
x.size()

torch.Size([2, 5, 512])

In [ ]:
qkv_layer = nn.Linear(in_dim,out_dim*3)
qkv = qkv_layer(x)
qkv.size()

torch.Size([2, 5, 1536])

In [ ]:
heads = 8
head_dim = in_dim//8
qkv = qkv.reshape(batch_size,seq,heads,head_dim*3)
qkv.size() # batch size, no of words, no of attention heads, each attention head dimension

torch.Size([2, 5, 8, 192])

In [ ]:
qkv = qkv.permute(0,2,1,3)
qkv.size()

torch.Size([2, 8, 5, 192])

In [ ]:
q,k,v = qkv.chunk(3,dim = -1)
q.size(),k.size(),v.size()

(torch.Size([2, 8, 5, 64]),
 torch.Size([2, 8, 5, 64]),
 torch.Size([2, 8, 5, 64]))

In [ ]:
scaled = torch.matmul(q,k.transpose(-2,-1))/math.sqrt(q.size()[-1])
scaled.size()

torch.Size([2, 8, 5, 5])

In [ ]:
attention = F.softmax(scaled,dim = -1)

In [ ]:
values = torch.matmul(attention,v)
values.size()

torch.Size([2, 8, 5, 64])

In [ ]:
values = values.reshape(batch_size,seq,heads*head_dim)
values.size()

torch.Size([2, 5, 512])

In [ ]:
import torch
import math
from torch import nn
import torch.nn.functional as F

In [ ]:
text = 'మళ్లీ ఉదయిస్తాడు.'
list(text)

['మ',
 'ళ',
 '్',
 'ల',
 'ీ',
 ' ',
 'ఉ',
 'ద',
 'య',
 'ి',
 'స',
 '్',
 'త',
 'ా',
 'డ',
 'ు',
 '.']

In [ ]:
# class Encoder(nn.Module):
  # def __init(self,num_layers,d_model,num_heads,ffn_hidden,drop_prob):

In [ ]:
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 20
ffn_hidden = 1024
num_layers = 2

# encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)